In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import re
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

driverPath = "chromedriver.exe" #填入剛剛記下的路徑
driver = webdriver.Chrome(driverPath) # Chrome
driver.implicitly_wait(30)
driver.get("https://www.yes123.com.tw/admin/index.asp")
search_input = driver.find_element_by_name("find_key1")
search_input.send_keys('富邦人壽')
start_search_btn = driver.find_element_by_class_name("bt_search")
start_search_btn.click()
condition = EC.visibility_of_element_located((By.CLASS_NAME,'sift'))
WebDriverWait(driver, 20, 0.5).until(condition)
soup = BeautifulSoup(driver.page_source,"html.parser")
s = re.findall("第.+頁",soup.text)[0]
totalpage = int(re.findall('\d+',s)[1])
current = int(re.findall('\d+',s)[0])

url_list=[]
for page in range(1,totalpage+1):
    print('page:'+str(page))
    while(current!=page):
        condition = EC.presence_of_element_located((By.NAME,"jump_page_num"))
        WebDriverWait(driver, 20, 0.5).until(condition)
        jump_page = driver.find_element_by_name("jump_page_num")
        jump_page.send_keys(page)
        
        condition = EC.element_to_be_clickable((By.CLASS_NAME,"bt_page"))
        WebDriverWait(driver, 20, 0.5).until(condition)
        jump_page_btn = driver.find_element_by_class_name("bt_page")
        jump_page_btn.click()
        
        condition = EC.visibility_of_element_located((By.CLASS_NAME,'sift'))
        WebDriverWait(driver, 20, 0.5).until(condition)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        s = re.findall("第.+頁",soup.text)[0]
        current = int(re.findall('\d+',s)[0])

    for i in soup.find_all(class_='jobname'):
        if i.get('href'):
            url_list.append('https://www.yes123.com.tw/admin/'+i['href'])
            
            
result=[]
for url in url_list:
    print(url)
    driver.get(url)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,"comp_detail"))
    WebDriverWait(driver, 20, 0.5).until(condition)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    url_company ='https://www.yes123.com.tw/admin/job_refer_comp_info.asp?'+re.findall('p_id.+&',url)[0][:-1]
    driver.get(url_company)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,'company_title'))
    WebDriverWait(driver, 20, 0.5).until(condition)
    soup_c = BeautifulSoup(driver.page_source,"html.parser")
    company = soup_c.find_all(class_='company_title')[0].text
    result.append([url,soup,company])
    sleep(0.3)

    
driver.close()

def getcontent(soup):
    soup_ul = soup.find_all('ul')
    sub_title = []
    sub_content = []
    for i in soup_ul:
        tt = i.find_all(class_="tt")
        rr = i.find_all(class_="rr")
        if tt!=[] and rr!=[]:
            for j in range(len(tt)):
                sub_title.append(tt[j].text.replace(' ： ',''))
                sub_content.append(rr[j].text.replace('\n','').replace('\t','').replace('\xa0','').replace('每月薪資行情表我要申訴','').replace(' ','').replace('地圖','').replace('就業導航',''))
    return dict(zip(sub_title,sub_content))

colname = ['web','url','工作名稱','薪資','上班地點','上班時段','公司','工作性質','聯絡人']
data=[]
for row in result:
    content = getcontent(row[1])
    value = ['yes123',
             row[0],
             row[1].find_all('h1')[0].text,
             content['薪資待遇'],
             content['工作地點'],
             content['上班時段'],
             row[2],
             content['工作性質'],
             content['連絡人'] ]
    r = dict(zip(colname,value))
    data.append(r)
df = pd.DataFrame(data)

page:1
page:2
page:3
page:4
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20120817114503_27935073&job_id=20191120131247_81582534
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20160620151643_27935073&job_id=20160620174002_23415406
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20160107143801_27935073&job_id=20160107195751_180289
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20131105174737_27935073&job_id=20131107103232_35839
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20200220151515_27935073&job_id=20200220155554_10620646
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20190703115447_27935073&job_id=20190712151906_65424631
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20120131144420_27935073&job_id=20190624173415_16071609
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20180713162604_27935073&job_id=20180713164506_15

https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20091008173295_27935073&job_id=20200103095627_161029
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20190122142617_27935073&job_id=20190123110209_1213150
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20180608114147_27935073&job_id=20180610231717_6118292
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20160721155211_27935073&job_id=20160722105524_24462369
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20160721155211_27935073&job_id=20160722143741_3384587
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20191029175028_27935073&job_id=20191030104449_3884933
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20160802165510_27935073&job_id=20160805115459_11356750
https://www.yes123.com.tw/admin/job_refer_comp_job_detail2.asp?p_id=20191002095652_27935073&job_id=20191002144425_23758112
https://www.yes123.com

In [2]:
df.shape

(94, 9)

In [ ]:
driver = webdriver.Chrome(driverPath) # Chrome
for url in url_list:
    print(url)
    driver.get(url)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,"comp_detail"))
    WebDriverWait(driver, 20, 0.5).until(condition)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    url_company ='https://www.yes123.com.tw/admin/job_refer_comp_info.asp?'+re.findall('p_id.+&',url)[0][:-1]
    driver.get(url_company)
    condition = EC.visibility_of_element_located((By.CLASS_NAME,'company_title'))
    driver.get(url_company)
    soup_c = BeautifulSoup(driver.page_source,"html.parser")
    company = soup_c.find_all(class_='company_title')[0].text
    result.append([url,soup,company])

In [ ]:
BeautifulSoup(driver.page_source,"html.parser")

In [ ]:
driver.refresh()